# Trace with LangChain

## Quick start

In [3]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

When we are using Ollama, logging a trace is optional. 

In [6]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user's request only based on the given context."),
    ("user", "Question: {question}\nContext: {context}")
])

model = ChatOllama(model='llama3.1')
output_parser = StrOutputParser()

chain = prompt | model | output_parser

"What a remarkable day it has been! This morning, leaders from around the globe gathered in a historic summit to address and resolve all existing world conflicts. After intense discussions and constructive negotiations, they were able to reach a comprehensive agreement that brought peace and harmony to every nation.\n\nThe meeting was attended by top diplomats, representatives from international organizations, and key stakeholders from over 100 countries. The atmosphere was collaborative and solution-focused, with a shared commitment to creating a more peaceful and prosperous world.\n\nKey highlights from the morning's meetings include:\n\n* A unified declaration against war and violence\n* Establishment of a new global framework for conflict resolution\n* Agreement on economic cooperation and trade partnerships\n* Commitment to environmental sustainability and climate action\n\nThe summit was a resounding success, marking a significant turning point in human history. As we move forwar

There're 2 ways.
1. Manually passing in a `LangChainTracer`
2. Using the `tracing_v2_enabled`

In [7]:
from langchain.callbacks.tracers import LangChainTracer

# 1. Instance as a callback
tracer = LangChainTracer()
chain.invoke({"question": "Am I using a callback?", "context": "I'm using a callback"}, config={"callbacks": [tracer]})

# 2. Context manager
from langchain_core.tracers.context import tracing_v2_enabled
with tracing_v2_enabled():
    chain.invoke({"question": "Am I using a context manager?", "context": "I'm using a context manager"})

# This will NOT be traced
chain.invoke({"question": "Am I being traced?", "context": "I'm not being traced"})

"No, you're not being traced."

## Log to a specific project

In [10]:
from langchain.callbacks.tracers import LangChainTracer

tracer = LangChainTracer(project_name="LangGraph Tutorial")
chain.invoke({"question": "Am I using a callback?", "context": "I'm using a callback"}, config={"callbacks": [tracer]})

from langchain_core.tracers.context import tracing_v2_enabled
with tracing_v2_enabled(project_name="LangGraph Tutorial"):
    chain.invoke({"question": "Am I using a context manager?", "context": "I'm using a context manager"})


## Add metadata and tags to traces

**NOTE**

When you attach metadata or tags to a runnable, they are inherited by all child runnables of that runnable.



In [6]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI."),
  ("user", "{input}")
])

chat_model = ChatOllama(model='llama3.1').with_config({"tags": ["config-tag"], "metadata": {"config-key": "config-value"}})
output_parser = StrOutputParser()

chain = (prompt | chat_model | output_parser).with_config({"tags": ["config-tag"], "metadata": {"config-key": "config-value"}})

# Tags and metadata can also be passed at runtime
chain.invoke({"input": "What is the meaning of life?"}, {"tags": ["invoke-tag"], "metadata": {"invoke-key": "invoke-value"}})

"One of the most profound and enduring questions of all time!\n\nThe meaning of life is a complex and multifaceted concept that has been debated, explored, and pondered by philosophers, theologians, scientists, and thinkers across cultures and centuries. While there may not be a single, definitive answer, here are some insights to consider:\n\n1. **Subjective experiences**: For many people, the meaning of life is closely tied to their personal values, goals, and aspirations. It's about pursuing happiness, fulfillment, and purpose through relationships, work, hobbies, or spiritual practices.\n2. **Biological imperatives**: From a biological perspective, the meaning of life might be seen as ensuring the survival and propagation of our species. This view emphasizes the importance of reproduction, adaptation, and the perpetuation of genetic information.\n3. **Existentialism**: Existentialist philosophers like Jean-Paul Sartre and Albert Camus suggest that life has no inherent meaning; inst

## Customize run name

**NOTE**

`run_name` is not currently supported directly for LLM objects.

In [8]:
configured_chain = chain.with_config({"run_name": "MyCustomChain"})
configured_chain.invoke({"input": "What is the meaning of life?"})


chain.invoke({"input": "What is the meaning of life?"}, {"run_name":"MyCustomChain"})

"The question of the meaning of life has been debated and explored by philosophers, theologians, scientists, and thinkers across various cultures and disciplines for centuries. While there might not be a single definitive answer, here's a nuanced perspective:\n\n**Philosophical perspectives:**\n\n1. **Existentialism:** This philosophy suggests that life has no inherent meaning; instead, individuals must create their own purpose and meaning through experiences, choices, and relationships.\n2. **Humanistic existentialism:** Building on existentialism, this approach emphasizes the importance of personal growth, self-actualization, and finding one's place in society.\n3. **Absurdism:** This philosophy acknowledges that life is inherently meaningless but proposes to make the most of it by embracing the absurdity and finding creative ways to cope with it.\n\n**Scientific perspectives:**\n\n1. **Evolutionary theory:** From a biological perspective, life on Earth has evolved to optimize surviv

## Access run(span) ID for Langchain invocations

`collect_runs`: Access the run ID.

In [9]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tracers.context import collect_runs

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user's request only based on the given context."),
    ("user", "Question: {question}\n\nContext: {context}")
])
model = ChatOllama(model="llama3.1")
output_parser = StrOutputParser()

chain = prompt | chain | output_parser

question = "Can you summarize this morning's meetings?"
context = "During this morning's meeting, we solved all world conflict."
with collect_runs() as cb:
    result = chain.invoke({"question": question, "context": context})
    # Get the root run id
    run_id = cb.traced_runs[0].id

print(run_id)

69fa387b-d412-47e9-a614-144637348f16


## Ensure all traces are submitted before existing

In [10]:
from langchain_ollama import ChatOllama
from langchain_core.tracers.langchain import wait_for_all_tracers

llm = ChatOllama(model="llama3.1")
try:
    llm.invoke("Hello, World!")
finally:
    wait_for_all_tracers()

## Trace without setting environment variables
When it is not possible to set environments variables, set the tracing configuration programmatically

In [11]:
from langchain.callbacks.tracers import LangChainTracer
from langsmith import Client

# Create a client instance with an api key and api url
client = Client(
    api_key="...",
    api_url="https://api.smith.langchain.com"
)

# Pass the client and project_name to the LangChainTracer instance
tracer = LangChainTracer(client=client, project_name="LangGraph Tutorial")
chain.invoke({"question": "Am I using a callback?", "context": "I'm using a callback"}, config={"callbacks": [tracer]})

# 
from langchain_core.tracers.context import tracing_v2_enabled
with tracing_v2_enabled(client=client, project_name="LangGraph Tutorial"):
    chain.invoke({"question": "Am I using a context manager?", "context": "I'm using a context manager"})